In [289]:
import yahoo_fin.stock_info as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [290]:
nasdaq_ticker = pd.read_csv('./data/nasdaq_screener.csv')
nasdaq_ticker.sort_values(['Market Cap'],ascending=False)
fifty_largest= nasdaq_ticker.nlargest(50,'Market Cap')
twenty_largest= nasdaq_ticker.nlargest(20,'Market Cap')
ticker_list = twenty_largest['Symbol'].to_list()


In [291]:
def get_company_financials(ticker_list):
    all_financials = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        # avoid looping through dataframes due to loc index issues
        for tick in ticker_list:
            ticker_report ={}
            dict_financials  = yf.get_financials(tick,yearly =True, quarterly = False)
            ticker_report["income_statement"]= dict_financials['yearly_income_statement'].fillna(0)
            ticker_report["balance_sheet"] = dict_financials['yearly_balance_sheet'].fillna(0)
            ticker_report["yearly_cash_flow"] = dict_financials['yearly_cash_flow'].fillna(0)
            all_financials[tick] = ticker_report
    return all_financials

In [292]:
def get_company_performance(ticker_list):
    all_performance = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        for tick in ticker_list:
            performance = {}

            try:
                dividend = yf.get_dividends(tick)
                performance["dividend"] = dividend
                earnings = yf.get_earnings_history(tick)
                performance["earnings"] = pd.DataFrame(earnings)
                all_performance[tick] = performance
                # catch exceptions raised on no data available
            except Exception as e : 
                ticker_list.remove(tick)
                #print(f"No dividend for {tick}") 
                
    return all_performance  



In [293]:
all_financials = get_company_financials(ticker_list)
all_performance = get_company_performance(ticker_list)

In [294]:
list_dfs_FCFE  = []
list_dfs_ND2EB = []
cf_labels = ['netBorrowings','capitalExpenditures','depreciation']
is_labels = ["netIncome",'ebit']

df_ratios = pd.DataFrame()
for tick in all_financials.keys():
    stock_ratios ={}
    yearly_cash_flow = all_financials[tick]["yearly_cash_flow"]
    income_stmnt = all_financials[tick]["income_statement"]
    if (cf_labels[0] in yearly_cash_flow.index) and (cf_labels[1] in yearly_cash_flow.index):
         capex = yearly_cash_flow.loc[cf_labels[1]]
         net_borrow = yearly_cash_flow.loc[cf_labels[0]]
         depreciation = yearly_cash_flow.loc[cf_labels[2]]
    else: 
        #print(tick)
        del all_financials[tick]    
    if is_labels[0] in income_stmnt.index:
        net_income = income_stmnt.loc["netIncome"]
        ebit = income_stmnt.loc["ebit"]
    else: 
        #print(tick)
        del all_financials[tick]
    FCFE = net_income - capex + net_borrow
    ND_to_EB = net_borrow / (ebit + depreciation)
    #print(f"Ticker {tick} has FCFE of {FCFE}")
    df_FCFE = pd.DataFrame(FCFE).transpose()
    df_ND2EB = pd.DataFrame(ND_to_EB).transpose()
    
    df_ND2EB.columns = df_ND2EB.columns.year
    df_ND2EB['Ticker'] = pd.Series(tick, index=df_ND2EB.index)
    list_dfs_ND2EB.append(df_ND2EB)
    
    df_FCFE.columns = df_FCFE.columns.year
    df_FCFE['Ticker'] = pd.Series(tick, index=df_FCFE.index)
    list_dfs_FCFE.append(df_FCFE)
   

In [230]:
df_all_ND2EB = pd.concat(list_dfs_ND2EB).set_index("Ticker").drop(columns=(min(df_all_ND2EB.columns) + 4),axis=1)
df_all_FCFE = pd.concat(list_dfs_FCFE).set_index("Ticker").drop(columns=(min(df_all_FCFE.columns) + 4),axis=1)

In [295]:
df_all_ND2EB.columns.name= "year"
df_all_FCFE.columns.name= "year"

In [296]:
calc_years= df_all_FCFE.columns
type(calc_years)

pandas.core.indexes.base.Index

In [297]:
all_div_payout= []
all_div_coverage=[]

for tick in all_performance.keys():
    
    divi = all_performance[tick]['dividend']
    ann_total_divi = divi.groupby(divi.index.year).sum()
    ann_total_divi = ann_total.T
    ann_total_divi = ann_total_divi[ann_total_divi.columns.intersection(df_all_FCFE.columns)]
   
    earnings = all_performance[tick]['earnings']
    eps = earnings[["startdatetime","epsactual"]]
    eps.loc[:, ("startdatetime")] = pd.to_datetime(eps.loc[:, ("startdatetime")], errors="coerce",format="%Y-%m-%d")
    eps_annual = eps.groupby(eps.loc[:, ("startdatetime")].dt.year).sum()
    eps_annual = eps_annual.T
    eps_annual = eps_annual[eps_annual.columns.intersection(df_all_FCFE.columns)]
    div_coverage = pd.DataFrame(eps_annual.loc['epsactual'] / ann_total_divi.loc["dividend"]).T
    div_coverage["Ticker"] = tick
    div_coverage =div_coverage.set_index("Ticker")
    all_div_coverage.append(div_coverage)
   
    div_payout =  pd.DataFrame(ann_total_divi.loc["dividend"] / eps_annual.loc['epsactual']).T
    div_payout["Ticker"] = tick
    div_payout =div_payout.set_index("Ticker")
    all_div_payout.append(div_payout)
   

In [302]:
df_div_payout = pd.concat(all_div_payout)
df_div_payout.columns.name= "year"
df_div_coverage= pd.concat(all_div_coverage)
df_div_coverage.columns.name= "year"

In [303]:
df_div_payout

year,2017,2018,2019,2020
Ticker,,,,
AAPL,0.172826,0.177503,0.634228,0.639144
MSFT,0.470414,0.411483,0.378758,0.337641
ASML,0.307544,0.252941,0.309329,0.349498
CMCSA,0.787129,0.713693,0.636364,0.735915
CSCO,0.665272,0.627737,0.594340,0.669872
AVGO,0.099313,0.082573,0.088691,0.094399
PEP,0.310547,0.314442,0.339928,0.380000
TXN,0.384988,0.317343,0.351301,0.395085


In [304]:
df_div_coverage

year,2017,2018,2019,2020
Ticker,,,,
AAPL,5.786164,5.633721,1.576720,1.564593
MSFT,2.125786,2.430233,2.640212,2.961722
ASML,3.251572,3.953488,3.232804,2.861244
CMCSA,1.270440,1.401163,1.571429,1.358852
CSCO,1.503145,1.593023,1.682540,1.492823
AVGO,10.069182,12.110465,11.275132,10.593301
PEP,3.220126,3.180233,2.941799,2.631579
TXN,2.597484,3.151163,2.846561,2.531100


In [305]:
df_all_FCFE

year,2020,2019,2018,2017
Ticker,,,,
AAPL,6.721900e+10,5.793200e+10,7.327600e+10,8.981600e+10
MSFT,5.420400e+10,4.916500e+10,1.800200e+10,6.507700e+10
AMZN,6.036700e+10,1.838300e+10,1.581400e+10,2.491600e+10
GOOG,7.221100e+10,5.762300e+10,5.581400e+10,2.576000e+10
GOOGL,7.221100e+10,5.762300e+10,5.581400e+10,2.576000e+10
FB,4.368100e+10,3.281200e+10,3.652700e+10,2.316700e+10
TSLA,1.465000e+09,1.368000e+09,1.432000e+09,5.504000e+09
NVDA,8.253000e+09,4.725000e+09,2.828000e+09,NaN
PYPL,9.034000e+09,6.118000e+09,3.840000e+09,3.282000e+09


In [306]:
df_all_ND2EB

year,2020,2019,2018,2017
Ticker,,,,
AAPL,0.032310,-0.102240,0.005281,0.405785
MSFT,-0.084555,-0.073315,-0.226901,0.847244
AMZN,-0.022928,-0.278120,-0.276853,0.637064
GOOG,0.175965,-0.005568,-0.001466,-0.002401
GOOGL,0.175965,-0.005568,-0.001466,-0.002401
FB,-0.014671,-0.022317,0.017107,0.021526
TSLA,-0.582261,0.367403,0.054434,846.250000
NVDA,1.539510,-0.003935,-0.238193,NaN
PYPL,0.923399,0.866315,0.351262,0.302583
